In [1]:
import pandas as pd
import numpy as np
import json
import mmap

In [2]:
games = []
with open("../data/diplomacy-v1-27k-msgs/standard_no_press.jsonl", "r+b") as json_file:
    with mmap.mmap(json_file.fileno(), length=0, access=mmap.ACCESS_READ) as mmap_object:
        for i, line in enumerate(iter(mmap_object.readline, b"")):
            games.append(json.loads(line.decode("utf-8")))
            if i == 100:
                break

In [3]:
# Convert to a pandas dataframe
df = pd.DataFrame(games)

In [4]:
phases = df['phases']

In [5]:
phases.apply(lambda x: x[0])

0      {'name': 'S1901M', 'state': {'timestamp': 1542...
1      {'name': 'S1901M', 'state': {'timestamp': 1542...
2      {'name': 'S1901M', 'state': {'timestamp': 1542...
3      {'name': 'S1901M', 'state': {'timestamp': 1542...
4      {'name': 'S1901M', 'state': {'timestamp': 1542...
                             ...                        
96     {'name': 'S1901M', 'state': {'timestamp': 1542...
97     {'name': 'S1901M', 'state': {'timestamp': 1542...
98     {'name': 'S1901M', 'state': {'timestamp': 1542...
99     {'name': 'S1901M', 'state': {'timestamp': 1542...
100    {'name': 'S1901M', 'state': {'timestamp': 1542...
Name: phases, Length: 101, dtype: object

In [6]:
json_file_path = "../data/game_rl_0.json"

with open(json_file_path, 'r') as j:
     contents = json.loads(j.read())

In [7]:
# contents

In [8]:
type(contents)

dict

WORK ON ONE DICTIONARY/PHASE

In [9]:
data = contents['phases'][0]

In [10]:
type(data)

dict

In [11]:
#One dictionary
# data

In [12]:
data.keys()

dict_keys(['name', 'state', 'orders', 'results', 'messages'])

In [13]:
# for key, value in data.items():
#   print(key,value)

In [14]:
data['state']['units']

{'AUSTRIA': ['A BUD', 'A VIE', 'F TRI'],
 'ENGLAND': ['F EDI', 'F LON', 'A LVP'],
 'FRANCE': ['F BRE', 'A MAR', 'A PAR'],
 'GERMANY': ['F KIE', 'A BER', 'A MUN'],
 'ITALY': ['F NAP', 'A ROM', 'A VEN'],
 'RUSSIA': ['A WAR', 'A MOS', 'F SEV', 'F STP/SC'],
 'TURKEY': ['F ANK', 'A CON', 'A SMY']}

In [15]:
data['orders']

{'AUSTRIA': ['A VIE - TRI', 'F TRI - ALB', 'A BUD - SER'],
 'ENGLAND': ['F EDI - NTH', 'F LON - ENG', 'A LVP - YOR'],
 'FRANCE': ['F BRE - MAO', 'A PAR - BUR', 'A MAR - SPA'],
 'GERMANY': ['F KIE - DEN', 'A MUN - RUH', 'A BER - KIE'],
 'ITALY': ['F NAP - ION', 'A ROM - VEN', 'A VEN - TRI'],
 'RUSSIA': ['F STP/SC - BOT', 'A MOS - UKR', 'A WAR - GAL', 'F SEV - BLA'],
 'TURKEY': ['F ANK - BLA', 'A SMY - CON', 'A CON - BUL']}

In [16]:
data['results']

{'A BUD': [],
 'A VIE': ['bounce'],
 'F TRI': [],
 'F EDI': [],
 'F LON': [],
 'A LVP': [],
 'F BRE': [],
 'A MAR': [],
 'A PAR': [],
 'F KIE': [],
 'A BER': [],
 'A MUN': [],
 'F NAP': [],
 'A ROM': ['bounce'],
 'A VEN': ['bounce'],
 'A WAR': [],
 'A MOS': [],
 'F SEV': ['bounce'],
 'F STP/SC': [],
 'F ANK': ['bounce'],
 'A CON': [],
 'A SMY': []}

Nested dictionaries are:
1. state
2. orders
3. results

In [17]:
# for key, value in data['state'].items():
#   print(key, ':', value)

In [18]:
#Action of each player
# for key, value in data['orders'].items():
#   print(key, ':', value)

In [19]:
#Results
# for key, value in data['results'].items():
#   print(key, ':', value)

FINAL DF FOR COMPLETE DATASET

In [20]:
complete_data = contents['phases']

In [21]:
# type(complete_data)

In [22]:
# complete_data

In [23]:
for ix, iy in enumerate(complete_data):
    # print(complete_data[ix])
    complete_data[ix]['phase_id'] = ix
    # print(complete_data[ix])

In [24]:
# print(complete_data[0]['state']['units'])
# print(complete_data[0]['orders'])
# print(complete_data[0]['results'])

# print(complete_data[1]['state']['units'])
# print(complete_data[1]['orders'])
# print(complete_data[1]['results'])

In [25]:
# complete_data[1]

In [73]:
def flatten_json_4(nested_json):
    out = {}
    x = nested_json

    out['game_id'] = x['state']['game_id']
    out['phase_id'] = x['phase_id']
    results_units = [x for x in x['results']]
    # print("results are: ", results_units)
    for key, value in x['orders'].items():
        if value == None:
            out['coordinator'] = key
            out['type'] = 'N'
            out['current_location'] = 'N'
            out['action'] = 'N'
            out['results'] = 'N'
            out['via'] = 'N'
            out['impact_unit_type'] = 'N'
            out['impact_location'] = 'N'
            yield(out)
        else:
            for ix in value:
                out['coordinator'] = key
                out['type'] = ix.split()[0]
                out['current_location'] = ix.split()[1]
                out['action'] = ix.split()[2]
                unit = ix.split()[0]+' ' + ix.split()[1]
                out['results'] = x['results'][unit]
                # print("unit result is: ", unit, out['results'], unit in results_units)
                if unit in results_units:
                    results_units.remove(unit)
                out['via'] = []
                out['impact_unit_type'] = []
                out['impact_location'] = []
                if len(ix.split()[3:])==1:
                    out['impact_location'] = ix.split()[3]
                elif len(ix.split()[3:])==2:
                    if ix.split()[-1]=='VIA':
                        out['impact_location'] = ix.split()[3]
                        out['via'] = ix.split()[-1]
                    else:
                        out['impact_unit_type'] = ix.split()[3]
                        out['impact_location'] = ix.split()[4]  
                elif len(ix.split()[3:])==4:                             
                    out['impact_unit_type'] = ix.split()[3]
                    out['impact_location'] = ix.split()[4]
                yield(out)

    if len(results_units)>0:
        # print("adding new rows: ", results_units)
        for ix in results_units:
            out['coordinator'] = 'N'
            for key in x["state"]["units"]:
                if ix in x["state"]["units"][key]:
                    # print(key, value, "found")
                    out['coordinator'] = key
            out['type'] = ix.split()[0] if len(ix.split()) > 1 else 'N'
            out['current_location'] = ix.split()[1] if len(ix.split()) > 1 else ix
            out['action'] = 'N'
            out['results'] = x['results'][ix]
            out['via'] = 'N'
            out['impact_unit_type'] = 'N'
            out['impact_location'] = 'N'
            yield(out)


In [74]:
complete_df = pd.DataFrame()
for i in complete_data:
  row_i = flatten_json_4(i)
  if row_i != None:
    for ix in row_i:  
      row_df = pd.DataFrame(ix.items()).T
      complete_df = pd.concat((complete_df,row_df), axis=0, ignore_index=True)

new_header = complete_df.iloc[0] #grab the first row for the header
complete_df = complete_df[1:] #take the data less the header row
complete_df.columns = new_header #set the header row as the df header
complete_df = complete_df.loc[::2]
complete_df = complete_df.reset_index()
complete_df = complete_df.drop('index', axis=1)

In [113]:
complete_df

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
0,ON_23cPO0RhktywJ,0,AUSTRIA,A,VIE,-,[bounce],[],[],TRI,-1
1,ON_23cPO0RhktywJ,0,AUSTRIA,F,TRI,-,[],[],[],ALB,-1
2,ON_23cPO0RhktywJ,0,AUSTRIA,A,BUD,-,[],[],[],SER,-1
3,ON_23cPO0RhktywJ,0,ENGLAND,F,EDI,-,[],[],[],NTH,-1
4,ON_23cPO0RhktywJ,0,ENGLAND,F,LON,-,[],[],[],ENG,-1
...,...,...,...,...,...,...,...,...,...,...,...
585,ON_23cPO0RhktywJ,34,FRANCE,N,N,N,N,N,N,N,-1
586,ON_23cPO0RhktywJ,34,GERMANY,N,N,N,N,N,N,N,-1
587,ON_23cPO0RhktywJ,34,ITALY,N,N,N,N,N,N,N,-1
588,ON_23cPO0RhktywJ,34,RUSSIA,N,N,N,N,N,N,N,-1


In [76]:
complete_df.loc[50:80]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location
50,ON_23cPO0RhktywJ,2,RUSSIA,A,STP,B,[],[],[],[]
51,ON_23cPO0RhktywJ,2,RUSSIA,A,MOS,B,[],[],[],[]
52,ON_23cPO0RhktywJ,2,RUSSIA,A,WAR,B,[],[],[],[]
53,ON_23cPO0RhktywJ,2,TURKEY,F,SMY,B,[],[],[],[]
54,ON_23cPO0RhktywJ,3,AUSTRIA,A,TRI,-,"[bounce, dislodged]",[],[],VIE
55,ON_23cPO0RhktywJ,3,AUSTRIA,F,GRE,-,[bounce],[],[],AEG
56,ON_23cPO0RhktywJ,3,AUSTRIA,A,BUD,S,[cut],[],A,TRI
57,ON_23cPO0RhktywJ,3,AUSTRIA,A,SER,S,[],[],A,BUL
58,ON_23cPO0RhktywJ,3,ENGLAND,F,LVP,-,[],[],[],IRI
59,ON_23cPO0RhktywJ,3,ENGLAND,F,NTH,S,[],[],A,NWY


In [77]:
complete_df[complete_df['phase_id'] == 3]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location
54,ON_23cPO0RhktywJ,3,AUSTRIA,A,TRI,-,"[bounce, dislodged]",[],[],VIE
55,ON_23cPO0RhktywJ,3,AUSTRIA,F,GRE,-,[bounce],[],[],AEG
56,ON_23cPO0RhktywJ,3,AUSTRIA,A,BUD,S,[cut],[],A,TRI
57,ON_23cPO0RhktywJ,3,AUSTRIA,A,SER,S,[],[],A,BUL
58,ON_23cPO0RhktywJ,3,ENGLAND,F,LVP,-,[],[],[],IRI
59,ON_23cPO0RhktywJ,3,ENGLAND,F,NTH,S,[],[],A,NWY
60,ON_23cPO0RhktywJ,3,ENGLAND,F,ENG,-,[bounce],[],[],MAO
61,ON_23cPO0RhktywJ,3,ENGLAND,A,NWY,S,[],[],F,DEN
62,ON_23cPO0RhktywJ,3,FRANCE,F,BRE,-,[bounce],[],[],MAO
63,ON_23cPO0RhktywJ,3,FRANCE,A,BUR,-,[bounce],[],[],RUH


In [78]:
complete_df[complete_df['phase_id'] == 21]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location
394,ON_23cPO0RhktywJ,21,ENGLAND,F,DEN,R,[],[],[],HEL
395,ON_23cPO0RhktywJ,21,RUSSIA,A,BUD,R,[],[],[],SER
396,ON_23cPO0RhktywJ,21,ITALY,F,BUL/SC,N,[disband],N,N,N


In [79]:
complete_df.index[complete_df['current_location'] == 'TRI'].tolist()

[1, 54, 86, 109, 127, 161, 188, 228, 264, 307, 340, 382, 418, 463, 497, 573]

In [80]:
initial_units = complete_df.current_location[complete_df['phase_id'] == 0].tolist()

In [81]:
print(complete_df.current_location.value_counts())

BUD       22
RUM       18
VIE       17
SMY       16
NTH       16
          ..
PRU        1
WAIVE      1
SPA/SC     1
NAO        1
YOR        1
Name: current_location, Length: 71, dtype: int64


In [82]:
complete_df['unique_unit_id'] = -1

In [83]:
complete_df[:50]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
0,ON_23cPO0RhktywJ,0,AUSTRIA,A,VIE,-,[bounce],[],[],TRI,-1
1,ON_23cPO0RhktywJ,0,AUSTRIA,F,TRI,-,[],[],[],ALB,-1
2,ON_23cPO0RhktywJ,0,AUSTRIA,A,BUD,-,[],[],[],SER,-1
3,ON_23cPO0RhktywJ,0,ENGLAND,F,EDI,-,[],[],[],NTH,-1
4,ON_23cPO0RhktywJ,0,ENGLAND,F,LON,-,[],[],[],ENG,-1
5,ON_23cPO0RhktywJ,0,ENGLAND,A,LVP,-,[],[],[],YOR,-1
6,ON_23cPO0RhktywJ,0,FRANCE,F,BRE,-,[],[],[],MAO,-1
7,ON_23cPO0RhktywJ,0,FRANCE,A,PAR,-,[],[],[],BUR,-1
8,ON_23cPO0RhktywJ,0,FRANCE,A,MAR,-,[],[],[],SPA,-1
9,ON_23cPO0RhktywJ,0,GERMANY,F,KIE,-,[],[],[],DEN,-1


# Temporal Binding

In [100]:
def assign_unit_id(phase_df, source_unit_id_map, dest_unit_id_map, _id):

    # fror each row in the phase df
    for idx, row in phase_df.iterrows():

        # if current location or type of army in invalid skip the row (we only deal with valid orders)
        if row['current_location'] == 'N' or row['type'] == 'N':
            continue
        
        # get the current location of the unit
        source_unit = row['type'] + ' ' + row['current_location']

        # if the location is not in the map, add it to the map (in other phases the same unit can be used, hence checking the condition _ dictionaries are global, have data across phases)
        if source_unit not in source_unit_id_map:
            source_unit_id_map[source_unit] = _id
            _id += 1

        # destination dict is synced with source dict after the loop, so that we can use updated info at the beginning of each assignment
        row['unique_unit_id'] = source_unit_id_map[source_unit]


        # print(idx, row)

        if row['action'] == '-':
            result = row['results']
            if isinstance(result, list):
                if len(result) == 0:
                    dest_location = row['impact_location']
                    dest_unit = row['type'] + ' ' + dest_location
                    if dest_unit not in dest_unit_id_map:
                        dest_unit_id_map[dest_unit] = source_unit_id_map[source_unit]
                        source_unit_id_map.pop(source_unit)
                elif 'disband' in result:
                    source_unit_id_map.pop(source_unit)

        elif row['action'] == 'R':
            result = row['results']
            if isinstance(result, list) and len(result) == 0:
                if len(result) == 0:
                    dest_location = row['impact_location']
                    dest_unit = row['type'] + ' ' + dest_location
                    if dest_unit not in dest_unit_id_map:
                        source_unit_id_map.pop(source_unit)
                elif 'disband' in result:
                    source_unit_id_map.pop(source_unit)

        elif row['action'] == 'D':
            source_unit_id_map.pop(source_unit)
        
        elif row['action'] == 'B':
            assert source_unit in source_unit_id_map

        phase_df.loc[idx] = row

    # merge the source and destination dictionaries into one
    source_unit_id_map.update(dest_unit_id_map)
    # remove the destination dict (values get updated based on old data if we don't do this)
    dest_unit_id_map = {}

    return phase_df, source_unit_id_map, dest_unit_id_map, _id


In [101]:
phases = complete_df["phase_id"].unique()

In [102]:
phase_df = complete_df[complete_df["phase_id"] == 0]

In [103]:
phase_df

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
0,ON_23cPO0RhktywJ,0,AUSTRIA,A,VIE,-,[bounce],[],[],TRI,-1
1,ON_23cPO0RhktywJ,0,AUSTRIA,F,TRI,-,[],[],[],ALB,-1
2,ON_23cPO0RhktywJ,0,AUSTRIA,A,BUD,-,[],[],[],SER,-1
3,ON_23cPO0RhktywJ,0,ENGLAND,F,EDI,-,[],[],[],NTH,-1
4,ON_23cPO0RhktywJ,0,ENGLAND,F,LON,-,[],[],[],ENG,-1
5,ON_23cPO0RhktywJ,0,ENGLAND,A,LVP,-,[],[],[],YOR,-1
6,ON_23cPO0RhktywJ,0,FRANCE,F,BRE,-,[],[],[],MAO,-1
7,ON_23cPO0RhktywJ,0,FRANCE,A,PAR,-,[],[],[],BUR,-1
8,ON_23cPO0RhktywJ,0,FRANCE,A,MAR,-,[],[],[],SPA,-1
9,ON_23cPO0RhktywJ,0,GERMANY,F,KIE,-,[],[],[],DEN,-1


In [104]:
s_dict = {}
d_dict = {}
_id = 1

In [105]:
phase_df_list = []
for phase in phases:
    print("phase number is", phase)
    phase_df = complete_df[complete_df["phase_id"] == phase]
    res_df, s_dict, d_dict, _id = assign_unit_id(phase_df, s_dict, d_dict, _id)
    phase_df_list.append(res_df)

phase number is 0
phase number is 1
phase number is 2
phase number is 3
phase number is 4
phase number is 5
phase number is 6
phase number is 7
phase number is 8
phase number is 9
phase number is 10
phase number is 11
phase number is 12
phase number is 13
phase number is 14
phase number is 15
phase number is 16
phase number is 17
phase number is 18
phase number is 19
phase number is 20
phase number is

/Users/ens/miniforge3/envs/marl/lib/python3.8/site-packages/pandas/core/indexing.py:1797: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, v, pi)


 21
phase number is 22
phase number is 23
phase number is 24
phase number is 25
phase number is 26
phase number is 27
phase number is 28
phase number is 29
phase number is 30
phase number is 31
phase number is 32
phase number is 33
phase number is 34


In [106]:
phase_df_list[0]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
0,ON_23cPO0RhktywJ,0,AUSTRIA,A,VIE,-,[bounce],[],[],TRI,1
1,ON_23cPO0RhktywJ,0,AUSTRIA,F,TRI,-,[],[],[],ALB,2
2,ON_23cPO0RhktywJ,0,AUSTRIA,A,BUD,-,[],[],[],SER,3
3,ON_23cPO0RhktywJ,0,ENGLAND,F,EDI,-,[],[],[],NTH,4
4,ON_23cPO0RhktywJ,0,ENGLAND,F,LON,-,[],[],[],ENG,5
5,ON_23cPO0RhktywJ,0,ENGLAND,A,LVP,-,[],[],[],YOR,6
6,ON_23cPO0RhktywJ,0,FRANCE,F,BRE,-,[],[],[],MAO,7
7,ON_23cPO0RhktywJ,0,FRANCE,A,PAR,-,[],[],[],BUR,8
8,ON_23cPO0RhktywJ,0,FRANCE,A,MAR,-,[],[],[],SPA,9
9,ON_23cPO0RhktywJ,0,GERMANY,F,KIE,-,[],[],[],DEN,10


In [107]:
phase_df_list[1]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
22,ON_23cPO0RhktywJ,1,AUSTRIA,A,VIE,-,[],[],[],TRI,1
23,ON_23cPO0RhktywJ,1,AUSTRIA,F,ALB,-,[],[],[],GRE,2
24,ON_23cPO0RhktywJ,1,AUSTRIA,A,SER,S,[],[],F,ALB,3
25,ON_23cPO0RhktywJ,1,ENGLAND,A,YOR,-,[],VIA,[],NWY,6
26,ON_23cPO0RhktywJ,1,ENGLAND,F,NTH,C,[],[],A,YOR,4
27,ON_23cPO0RhktywJ,1,ENGLAND,F,ENG,-,[bounce],[],[],BEL,5
28,ON_23cPO0RhktywJ,1,FRANCE,F,MAO,-,[],[],[],BRE,7
29,ON_23cPO0RhktywJ,1,FRANCE,A,BUR,-,[bounce],[],[],BEL,8
30,ON_23cPO0RhktywJ,1,FRANCE,A,SPA,-,[],[],[],POR,9
31,ON_23cPO0RhktywJ,1,GERMANY,F,DEN,H,[],[],[],[],10


In [108]:
phase_df_list[2]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
44,ON_23cPO0RhktywJ,2,AUSTRIA,A,BUD,B,[],[],[],[],23
45,ON_23cPO0RhktywJ,2,ENGLAND,F,LVP,B,[],[],[],[],24
46,ON_23cPO0RhktywJ,2,FRANCE,A,PAR,B,[],[],[],[],25
47,ON_23cPO0RhktywJ,2,GERMANY,A,KIE,B,[],[],[],[],26
48,ON_23cPO0RhktywJ,2,GERMANY,F,BER,B,[],[],[],[],27
49,ON_23cPO0RhktywJ,2,ITALY,F,NAP,B,[],[],[],[],28
50,ON_23cPO0RhktywJ,2,RUSSIA,A,STP,B,[],[],[],[],29
51,ON_23cPO0RhktywJ,2,RUSSIA,A,MOS,B,[],[],[],[],30
52,ON_23cPO0RhktywJ,2,RUSSIA,A,WAR,B,[],[],[],[],31
53,ON_23cPO0RhktywJ,2,TURKEY,F,SMY,B,[],[],[],[],32


In [109]:
phase_df_list[3]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
54,ON_23cPO0RhktywJ,3,AUSTRIA,A,TRI,-,"[bounce, dislodged]",[],[],VIE,1
55,ON_23cPO0RhktywJ,3,AUSTRIA,F,GRE,-,[bounce],[],[],AEG,2
56,ON_23cPO0RhktywJ,3,AUSTRIA,A,BUD,S,[cut],[],A,TRI,23
57,ON_23cPO0RhktywJ,3,AUSTRIA,A,SER,S,[],[],A,BUL,3
58,ON_23cPO0RhktywJ,3,ENGLAND,F,LVP,-,[],[],[],IRI,24
59,ON_23cPO0RhktywJ,3,ENGLAND,F,NTH,S,[],[],A,NWY,4
60,ON_23cPO0RhktywJ,3,ENGLAND,F,ENG,-,[bounce],[],[],MAO,5
61,ON_23cPO0RhktywJ,3,ENGLAND,A,NWY,S,[],[],F,DEN,6
62,ON_23cPO0RhktywJ,3,FRANCE,F,BRE,-,[bounce],[],[],MAO,7
63,ON_23cPO0RhktywJ,3,FRANCE,A,BUR,-,[bounce],[],[],RUH,8


In [110]:
cdf = df = pd.concat(phase_df_list)

In [111]:
cdf

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
0,ON_23cPO0RhktywJ,0,AUSTRIA,A,VIE,-,[bounce],[],[],TRI,1
1,ON_23cPO0RhktywJ,0,AUSTRIA,F,TRI,-,[],[],[],ALB,2
2,ON_23cPO0RhktywJ,0,AUSTRIA,A,BUD,-,[],[],[],SER,3
3,ON_23cPO0RhktywJ,0,ENGLAND,F,EDI,-,[],[],[],NTH,4
4,ON_23cPO0RhktywJ,0,ENGLAND,F,LON,-,[],[],[],ENG,5
...,...,...,...,...,...,...,...,...,...,...,...
585,ON_23cPO0RhktywJ,34,FRANCE,N,N,N,N,N,N,N,-1
586,ON_23cPO0RhktywJ,34,GERMANY,N,N,N,N,N,N,N,-1
587,ON_23cPO0RhktywJ,34,ITALY,N,N,N,N,N,N,N,-1
588,ON_23cPO0RhktywJ,34,RUSSIA,N,N,N,N,N,N,N,-1


In [124]:
with open('out.txt', 'w') as f:
    print(df.groupby(["game_id","coordinator","phase_id"])["unique_unit_id"].nunique().to_string(), file=f)

In [120]:
complete_df[complete_df['action'] == 'D']

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
124,ON_23cPO0RhktywJ,6,RUSSIA,F,SKA,D,[],[],[],[],-1
199,ON_23cPO0RhktywJ,10,AUSTRIA,A,ALB,D,[],[],[],[],-1
201,ON_23cPO0RhktywJ,10,FRANCE,A,POR,D,[],[],[],[],-1
281,ON_23cPO0RhktywJ,15,ENGLAND,F,WAL,D,[],[],[],[],-1
282,ON_23cPO0RhktywJ,15,ENGLAND,A,FIN,D,[],[],[],[],-1
354,ON_23cPO0RhktywJ,18,RUSSIA,A,STP,D,[disband],[],[],[],-1
432,ON_23cPO0RhktywJ,24,ENGLAND,F,LON,D,[],[],[],[],-1
433,ON_23cPO0RhktywJ,24,FRANCE,F,ENG,D,[],[],[],[],-1
438,ON_23cPO0RhktywJ,24,TURKEY,F,BLA,D,[],[],[],[],-1
513,ON_23cPO0RhktywJ,29,TURKEY,F,AEG,D,[],[],[],[],-1


# Manually check the first 50 rows

In [63]:
complete_df[:50]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
0,ON_23cPO0RhktywJ,0,AUSTRIA,A,VIE,-,[bounce],[],[],[TRI],1
1,ON_23cPO0RhktywJ,0,AUSTRIA,F,TRI,-,[],[],[],[ALB],2
2,ON_23cPO0RhktywJ,0,AUSTRIA,A,BUD,-,[],[],[],[SER],3
3,ON_23cPO0RhktywJ,0,ENGLAND,F,EDI,-,[],[],[],[NTH],4
4,ON_23cPO0RhktywJ,0,ENGLAND,F,LON,-,[],[],[],[ENG],5
5,ON_23cPO0RhktywJ,0,ENGLAND,A,LVP,-,[],[],[],[YOR],6
6,ON_23cPO0RhktywJ,0,FRANCE,F,BRE,-,[],[],[],[MAO],7
7,ON_23cPO0RhktywJ,0,FRANCE,A,PAR,-,[],[],[],[BUR],8
8,ON_23cPO0RhktywJ,0,FRANCE,A,MAR,-,[],[],[],[SPA],9
9,ON_23cPO0RhktywJ,0,GERMANY,F,KIE,-,[],[],[],[DEN],10


# IGNORE

In [62]:
     # while not done_flag:

    #     unit_df = complete_df[complete_df['current_location'] == unit]

    #     unit_moves = unit_df[unit_df['action'] == '-']
    #     unit_retreats = unit_df[unit_df['action'] == 'R']

    #     disband_actions = unit_df[unit_df['action'] == 'D']
    #     disband_results = unit_df[unit_df['results'].apply(lambda x: True if ('disband' in x) else False) == True]

    #     # unit_disbands = pd.concat((disband_actions, disband_results), axis=1, ignore_index=True)
    #     # print("unit_disbands is:", unit_disbands)
    #     # print("unit_moves are:", unit_moves)
    #     # print("unit_retreats are:", unit_retreats)
        
    #     valid_unit_moves = unit_moves.index[unit_moves['results'].apply(lambda x: True if isinstance(x, list) and len(x) == 0 else False)].tolist()
    #     valid_unit_retreats = unit_retreats.index[unit_retreats['results'].apply(lambda x: True if isinstance(x, list) and len(x) == 0 else False)].tolist()

    #     first_valid_unit_move = min(valid_unit_moves) if len(valid_unit_moves) > 0 else -1
    #     first_valid_unit_retreat = min(valid_unit_retreats) if len(valid_unit_retreats) > 0 else -1

    #     first_disband_action = min(disband_actions) if len(disband_actions) > 0 else -1
    #     first_disband_result = min(disband_results) if len(disband_results) > 0 else -1

    #     print("first_valid_unit_move is:", first_valid_unit_move)
    #     print("first_valid_unit_retreat is:", first_valid_unit_retreat)
    #     print("first_disband_action is:", first_disband_action)
    #     print("first_disband_result is:", first_disband_result)

    # if first_valid_unit_move != -1:
    #     print("moving unit:", unit)
    #     print("to:", unit_df.loc[first_valid_unit_move]['impact_location'][0])
    #     print("from:", unit_df.loc[first_valid_unit_move]['current_location'])

    # print("unit valid moves are :", valid_unit_moves)
    # print(complete_df[complete_df['action'] == 'D'])
    # print(complete_df[complete_df['action'] == 'B'])
    # print(complete_df[complete_df['results'].apply(lambda x: True if 'disband' in x else False) == True])

In [159]:
complete_df[complete_df['results'].apply(lambda x: True if ('disband' in x) else False) == True]

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location
88,ON_23cPO0RhktywJ,4,TURKEY,A,RUM,N,[disband],N,N,N
126,ON_23cPO0RhktywJ,6,RUSSIA,A,BUD,N,[disband],N,N,N
165,ON_23cPO0RhktywJ,8,RUSSIA,A,RUM,N,[disband],N,N,N
205,ON_23cPO0RhktywJ,10,ITALY,F,GRE,N,[disband],N,N,N
279,ON_23cPO0RhktywJ,14,ITALY,A,BUD,N,[disband],N,N,N
280,ON_23cPO0RhktywJ,14,RUSSIA,A,RUM,N,[disband],N,N,N
354,ON_23cPO0RhktywJ,18,RUSSIA,A,STP,D,[disband],[],[],[]
356,ON_23cPO0RhktywJ,18,RUSSIA,A,BUD,N,[disband],N,N,N
396,ON_23cPO0RhktywJ,21,ITALY,F,BUL/SC,N,[disband],N,N,N
431,ON_23cPO0RhktywJ,23,RUSSIA,A,BUD,N,[disband],N,N,N


In [160]:
complete_df[complete_df['action'] == 'D']

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location
124,ON_23cPO0RhktywJ,6,RUSSIA,F,SKA,D,[],[],[],[]
199,ON_23cPO0RhktywJ,10,AUSTRIA,A,ALB,D,[],[],[],[]
201,ON_23cPO0RhktywJ,10,FRANCE,A,POR,D,[],[],[],[]
281,ON_23cPO0RhktywJ,15,ENGLAND,F,WAL,D,[],[],[],[]
282,ON_23cPO0RhktywJ,15,ENGLAND,A,FIN,D,[],[],[],[]
354,ON_23cPO0RhktywJ,18,RUSSIA,A,STP,D,[disband],[],[],[]
432,ON_23cPO0RhktywJ,24,ENGLAND,F,LON,D,[],[],[],[]
433,ON_23cPO0RhktywJ,24,FRANCE,F,ENG,D,[],[],[],[]
438,ON_23cPO0RhktywJ,24,TURKEY,F,BLA,D,[],[],[],[]
513,ON_23cPO0RhktywJ,29,TURKEY,F,AEG,D,[],[],[],[]


In [162]:
complete_df[complete_df['action'] == 'B']

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location
44,ON_23cPO0RhktywJ,2,AUSTRIA,A,BUD,B,[],[],[],[]
45,ON_23cPO0RhktywJ,2,ENGLAND,F,LVP,B,[],[],[],[]
46,ON_23cPO0RhktywJ,2,FRANCE,A,PAR,B,[],[],[],[]
47,ON_23cPO0RhktywJ,2,GERMANY,A,KIE,B,[],[],[],[]
48,ON_23cPO0RhktywJ,2,GERMANY,F,BER,B,[],[],[],[]
49,ON_23cPO0RhktywJ,2,ITALY,F,NAP,B,[],[],[],[]
50,ON_23cPO0RhktywJ,2,RUSSIA,A,STP,B,[],[],[],[]
51,ON_23cPO0RhktywJ,2,RUSSIA,A,MOS,B,[],[],[],[]
52,ON_23cPO0RhktywJ,2,RUSSIA,A,WAR,B,[],[],[],[]
53,ON_23cPO0RhktywJ,2,TURKEY,F,SMY,B,[],[],[],[]


In [46]:
complete_df[complete_df['action'] == 'R']

,game_id,phase_id,coordinator,type,current_location,action,results,via,impact_unit_type,impact_location,unique_unit_id
86,ON_23cPO0RhktywJ,4,AUSTRIA,A,TRI,R,[],[],[],[ALB],-1
87,ON_23cPO0RhktywJ,4,RUSSIA,F,SWE,R,[],[],[],[SKA],-1
161,ON_23cPO0RhktywJ,8,AUSTRIA,A,TRI,R,[],[],[],[ALB],-1
162,ON_23cPO0RhktywJ,8,FRANCE,F,BRE,R,[],[],[],[GAS],-1
163,ON_23cPO0RhktywJ,8,FRANCE,A,BUR,R,[],[],[],[MAR],-1
164,ON_23cPO0RhktywJ,8,TURKEY,F,AEG,R,[],[],[],[CON],-1
240,ON_23cPO0RhktywJ,12,ENGLAND,F,NTH,R,[],[],[],[EDI],-1
241,ON_23cPO0RhktywJ,12,FRANCE,F,MAO,R,[],[],[],[POR],-1
242,ON_23cPO0RhktywJ,12,RUSSIA,A,RUM,R,[],[],[],[UKR],-1
277,ON_23cPO0RhktywJ,14,ENGLAND,A,NWY,R,[],[],[],[FIN],-1


---------------------------------------------------
-------------------------IGNORE FROM HERE ONWARDS-----------------------

---------------------------------------------------

In [ ]:
complete_df = pd.DataFrame()
for i in complete_data:
  row_i = flatten_json_4(i)
  for ix in row_i:
    print(ix)
  #row_df = pd.DataFrame.from_records(row_i, index=['A'])
  #row_df = pd.DataFrame(row_i.items()).T
  #row_df = pd.DataFrame.from_dict(row_i, orient='index')
  #print(type(row_df.loc[1]))
  #complete_df = pd.concat((complete_df,row_df), axis=0, ignore_index=True)

#new_header = complete_df.iloc[0] #grab the first row for the header
#complete_df = complete_df[1:] #take the data less the header row
#complete_df.columns = new_header #set the header row as the df header
#complete_df = complete_df.loc[::2]

{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'AUSTRIA_VIE', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'AUSTRIA_TRI', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'AUSTRIA_BUD', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'ENGLAND_EDI', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'ENGLAND_LON', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'ENGLAND_LVP', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'FRANCE_BRE', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'FRANCE_PAR', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'FRANCE_MAR', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'GERMANY_KIE', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'phase_id': 0, 'unit_id': 'GERMANY_MUN', 'action': '-'}
{'game_id': 'FPOF2T3X7Je6ogxb', 'ph

In [ ]:
complete_df = pd.DataFrame()
for i in complete_data:
  row_i = flatten_json_4(i)
  #row_df = pd.DataFrame.from_records(row_i, index=['A'])
  row_df = pd.DataFrame(row_i.items()).T
  #row_df = pd.DataFrame.from_dict(row_i, orient='index')
  #print(type(row_df.loc[1]))
  complete_df = pd.concat((complete_df,row_df), axis=0, ignore_index=True)

new_header = complete_df.iloc[0] #grab the first row for the header
complete_df = complete_df[1:] #take the data less the header row
complete_df.columns = new_header #set the header row as the df header
complete_df = complete_df.loc[::2]
complete_df = complete_df.reset_index()

In [ ]:
complete_df

,index,game_id,orders,phase_id
0,1,FPOF2T3X7Je6ogxb,"{'AUSTRIA': ['A VIE - GAL', 'F TRI - ALB', 'A ...",0
1,3,FPOF2T3X7Je6ogxb,"{'AUSTRIA': ['A VIE - TRI', 'F ALB - GRE', 'A ...",1
2,5,FPOF2T3X7Je6ogxb,"{'AUSTRIA': ['A BUD B'], 'ENGLAND': ['F LON B'...",2
3,7,FPOF2T3X7Je6ogxb,"{'AUSTRIA': ['A VIE S A BUD - TRI', 'F ALB - G...",3
4,9,FPOF2T3X7Je6ogxb,"{'AUSTRIA': ['A VIE - BUD', 'A TRI S A VIE - B...",4
...,...,...,...,...
67,135,FPOF2T3X7Je6ogxb,"{'AUSTRIA': [], 'ENGLAND': [], 'FRANCE': [], '...",67
68,137,FPOF2T3X7Je6ogxb,"{'AUSTRIA': [], 'ENGLAND': ['A EDI - NWY VIA',...",68
69,139,FPOF2T3X7Je6ogxb,"{'AUSTRIA': [], 'ENGLAND': ['A EDI - NWY VIA',...",69
70,141,FPOF2T3X7Je6ogxb,"{'AUSTRIA': [], 'ENGLAND': ['A SEV R MOS'], 'F...",70


PARSING INTO MULTIPLE ROWS

In [ ]:
from copy import copy

In [ ]:
complete_df_copy = copy(complete_df)

In [ ]:
complete_df_copy.iloc[0]['orders']

{'AUSTRIA': ['A VIE - GAL', 'F TRI - ALB', 'A BUD - SER'],
 'ENGLAND': ['F EDI - NWG', 'F LON - NTH', 'A LVP - YOR'],
 'FRANCE': ['F BRE - MAO', 'A PAR - BUR', 'A MAR - SPA'],
 'GERMANY': ['F KIE - DEN', 'A MUN - RUH', 'A BER - KIE'],
 'ITALY': ['F NAP - ION', 'A ROM - VEN', 'A VEN - TYR'],
 'RUSSIA': ['F STP/SC - BOT', 'A MOS - UKR', 'A WAR - GAL', 'F SEV - BLA'],
 'TURKEY': ['F ANK - BLA', 'A SMY - CON', 'A CON - BUL']}

In [ ]:
from pandas import Series

In [ ]:
s = complete_df_copy['orders'].str.split(' ').apply(Series, 1).stack()

In [ ]:
s.index = s.index.droplevel(-1) # to line up with df's index

In [ ]:
s.name = 'Seatblocks' # needs a name to join

In [ ]:
s

Series([], Name: Seatblocks, dtype: float64)

NOW WORKING ON THE WHOLE DATASET

In [ ]:
complete_data = contents['phases']

In [ ]:
for i, element in enumerate(complete_data):
  print("index is ", i)
  print("element is ", element)

index is  0
element is  {'name': 'S1901M', 'state': {'timestamp': 1644788746795152, 'zobrist_hash': '1919110489198082658', 'note': '', 'name': 'S1901M', 'units': {'AUSTRIA': ['A BUD', 'A VIE', 'F TRI'], 'ENGLAND': ['F EDI', 'F LON', 'A LVP'], 'FRANCE': ['F BRE', 'A MAR', 'A PAR'], 'GERMANY': ['F KIE', 'A BER', 'A MUN'], 'ITALY': ['F NAP', 'A ROM', 'A VEN'], 'RUSSIA': ['A WAR', 'A MOS', 'F SEV', 'F STP/SC'], 'TURKEY': ['F ANK', 'A CON', 'A SMY']}, 'retreats': {'AUSTRIA': {}, 'ENGLAND': {}, 'FRANCE': {}, 'GERMANY': {}, 'ITALY': {}, 'RUSSIA': {}, 'TURKEY': {}}, 'centers': {'AUSTRIA': ['BUD', 'TRI', 'VIE'], 'ENGLAND': ['EDI', 'LON', 'LVP'], 'FRANCE': ['BRE', 'MAR', 'PAR'], 'GERMANY': ['BER', 'KIE', 'MUN'], 'ITALY': ['NAP', 'ROM', 'VEN'], 'RUSSIA': ['MOS', 'SEV', 'STP', 'WAR'], 'TURKEY': ['ANK', 'CON', 'SMY']}, 'homes': {'AUSTRIA': ['BUD', 'TRI', 'VIE'], 'ENGLAND': ['EDI', 'LON', 'LVP'], 'FRANCE': ['BRE', 'MAR', 'PAR'], 'GERMANY': ['BER', 'KIE', 'MUN'], 'ITALY': ['NAP', 'ROM', 'VEN'], 'RUSS

In [ ]:
complete_df = pd.DataFrame()
for i in complete_data:
  row_i = flatten_json_2(i)
  #row_df = pd.DataFrame.from_records(row_i, index=['A'])
  row_df = pd.DataFrame(row_i.items()).T
  #row_df = pd.DataFrame.from_dict(row_i, orient='index')
  #print(type(row_df.loc[1]))
  complete_df = pd.concat((complete_df,row_df), axis=0, ignore_index=True)

new_header = complete_df.iloc[0] #grab the first row for the header
complete_df = complete_df[1:] #take the data less the header row
complete_df.columns = new_header #set the header row as the df header
complete_df = complete_df.loc[::2]

In [ ]:
complete_df

,name,state_timestamp,state_zobrist_hash,state_note,state_name,state_units_AUSTRIA,state_units_ENGLAND,state_units_FRANCE,state_units_GERMANY,state_units_ITALY,state_units_RUSSIA,state_units_TURKEY,state_centers_AUSTRIA,state_centers_ENGLAND,state_centers_FRANCE,state_centers_GERMANY,state_centers_ITALY,state_centers_RUSSIA,state_centers_TURKEY,state_homes_AUSTRIA,state_homes_ENGLAND,state_homes_FRANCE,state_homes_GERMANY,state_homes_ITALY,state_homes_RUSSIA,state_homes_TURKEY,state_influence_AUSTRIA,state_influence_ENGLAND,state_influence_FRANCE,state_influence_GERMANY,state_influence_ITALY,state_influence_RUSSIA,state_influence_TURKEY,state_civil_disorder_AUSTRIA,state_civil_disorder_ENGLAND,state_civil_disorder_FRANCE,state_civil_disorder_GERMANY,state_civil_disorder_ITALY,state_civil_disorder_RUSSIA,state_civil_disorder_TURKEY,...,orders_FRANCE,orders_GERMANY,orders_ITALY,orders_RUSSIA,orders_TURKEY,results_A BUD,results_A VIE,results_F TRI,results_F EDI,results_F LON,results_A LVP,results_F BRE,results_A MAR,results_A PAR,results_F KIE,results_A BER,results_A MUN,results_F NAP,results_A ROM,results_A VEN,results_A WAR,results_A MOS,results_F SEV,results_F STP/SC,results_F ANK,results_A CON,results_A SMY,messages,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,S1901M,1644788746795152,1919110489198082658,,S1901M,"[A BUD, A VIE, F TRI]","[F EDI, F LON, A LVP]","[F BRE, A MAR, A PAR]","[F KIE, A BER, A MUN]","[F NAP, A ROM, A VEN]","[A WAR, A MOS, F SEV, F STP/SC]","[F ANK, A CON, A SMY]","[BUD, TRI, VIE]","[EDI, LON, LVP]","[BRE, MAR, PAR]","[BER, KIE, MUN]","[NAP, ROM, VEN]","[MOS, SEV, STP, WAR]","[ANK, CON, SMY]","[BUD, TRI, VIE]","[EDI, LON, LVP]","[BRE, MAR, PAR]","[BER, KIE, MUN]","[NAP, ROM, VEN]","[MOS, SEV, STP, WAR]","[ANK, CON, SMY]","[BUD, VIE, TRI]","[EDI, LON, LVP]","[BRE, MAR, PAR]","[KIE, BER, MUN]","[NAP, ROM, VEN]","[WAR, MOS, SEV, STP]","[ANK, CON, SMY]",0,0,0,0,0,0,0,...,"[F BRE - MAO, A PAR - BUR, A MAR - SPA]","[F KIE - DEN, A MUN - RUH, A BER - KIE]","[F NAP - ION, A ROM - VEN, A VEN - TYR]","[F STP/SC - BOT, A MOS - UKR, A WAR - GAL, F S...","[F ANK - BLA, A SMY - CON, A CON - BUL]",[],[bounce],[],[],[],[],[],[],[],[],[],[],[],[],[],[bounce],[],[bounce],[],[bounce],[],[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F1901M,1644788747108210,8187135789066076135,,F1901M,"[A VIE, A SER, F ALB]","[F NWG, F NTH, A YOR]","[F MAO, A SPA, A BUR]","[F DEN, A KIE, A RUH]","[F ION, A VEN, A TYR]","[A WAR, F SEV, A UKR, F BOT]","[F ANK, A BUL, A CON]","[BUD, TRI, VIE]","[EDI, LON, LVP]","[BRE, MAR, PAR]","[BER, KIE, MUN]","[NAP, ROM, VEN]","[MOS, SEV, STP, WAR]","[ANK, CON, SMY]","[BUD, TRI, VIE]","[EDI, LON, LVP]","[BRE, MAR, PAR]","[BER, KIE, MUN]","[NAP, ROM, VEN]","[MOS, SEV, STP, WAR]","[ANK, CON, SMY]","[BUD, VIE, TRI, SER, ALB]","[EDI, LON, LVP, NWG, NTH, YOR]","[BRE, MAR, PAR, MAO, SPA, BUR]","[BER, MUN, DEN, KIE, RUH]","[NAP, ROM, ION, VEN, TYR]","[WAR, MOS, SEV, STP, UKR, BOT]","[ANK, SMY, BUL, CON]",0,0,0,0,0,0,0,...,"[F MAO - POR, A BUR - BEL, A SPA H]","[F DEN - SWE, A RUH - BEL, A KIE - HOL]","[A TYR S A VEN - TRI, F ION - TUN, A VEN - TRI]","[F BOT - SWE, A WAR - GAL, F SEV S A UKR - RUM...","[F ANK - BLA, A BUL - GRE, A CON - BUL]",[bounce],[],[bounce],[],[],[bounce],[],[],[bounce],[bounce],[],[bounce],[],[bounce],[],[],[],[],[bounce],[],[bounce],[bounce],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,W1901A,1644788747406895,6988686249571280341,,W1901A,"[A VIE, A SER, F ALB]","[F NTH, A YOR, F NWY]","[A SPA, A BUR, F POR]","[F DEN, A RUH, A HOL]","[A VEN, A TYR, F TUN]","[F SEV, F BOT, A GAL, A RUM]","[A BUL, A CON, F BLA]","[BUD, TRI, VIE, SER]","[EDI, LON, LVP, NWY]","[BRE, MAR, PAR, POR, SPA]","[BER, KIE, MUN, DEN, HOL]","[NAP, ROM, VEN, TUN]","[MOS, SEV, STP, WAR, RUM]","[ANK, CON, SMY, BUL]","[BUD, TRI, VIE]","[EDI, LON, LVP]","[BRE, MAR, PAR]","[BER, KIE, MUN]","[NAP, ROM, VEN]","[MOS, SEV, STP, WAR]","[ANK, CON, SMY]","[BUD, VIE, TRI, SER, ALB]","[EDI, LON, LVP, NWG, NTH, YOR,

In [ ]:
complete_df.columns

Index([                        'name',              'state_timestamp',
                 'state_zobrist_hash',                   'state_note',
                         'state_name',          'state_units_AUSTRIA',
                'state_units_ENGLAND',           'state_units_FRANCE',
                'state_units_GERMANY',            'state_units_ITALY',
                 'state_units_RUSSIA',           'state_units_TURKEY',
              'state_centers_AUSTRIA',        'state_centers_ENGLAND',
               'state_centers_FRANCE',        'state_centers_GERMANY',
                'state_centers_ITALY',         'state_centers_RUSSIA',
               'state_centers_TURKEY',          'state_homes_AUSTRIA',
                'state_homes_ENGLAND',           'state_homes_FRANCE',
                'state_homes_GERMANY',            'state_homes_ITALY',
                 'state_homes_RUSSIA',           'state_homes_TURKEY',
            'state_influence_AUSTRIA',      'state_influence_ENGLAND',
      

In [ ]:
def flatten_json_4(nested_json):
    out = {}
    x = nested_json
    #def flatten(x, name=''):
    #if type(x) is dict:
    for a in x.keys():
        #print(a)
        if a =='state':
            out['game_id'] = x[a]['game_id']
        if a == 'phase_id':
            #print('yes')
            out['phase_id'] = x[a]
            #print(out)
        if a== 'orders':
            #out['unit_id'] = []
            #out['action'] = []
            for iy, iz in x[a].items():
                if iz == None:
                    #out['unit_id'].append(iy + '_' + 'N')
                    out['unit_id'] = iy + '_' + 'N'
                    out['action'] = []
                    #print(out)
                    #yield(out)
                else:
                    for iw in iz:
                        #print(iw.split())
                        #out['unit_id'].append(iy + '_' + iw.split()[1])
                        out['unit_id'] = iy + '_' + iw.split()[1]
                        #out['action'].append(iw.split()[2])
                        out['action'] = iw.split()[2]
                        #print(out)
                        #yield(out)
          #for a in x:
              #flatten(x[a], name + a + '_')
      #elif type(x) is list:
      #    out[name[:-1]] = tuple(x)
      #elif type(x) is list:
      #    i = 0
      #    for a in x:
      #        flatten(a, name + str(i) + '_')
      #        i += 1
      #else:
          #out[name[:-1]] = x

  
  #flatten(nested_json)
  #return out

In [ ]:
def flatten_json_2(nested_json):
  out = {}

  def flatten(x, name=''):
      if type(x) is dict:
          for a in x:
              flatten(x[a], name + a + '_')
      #elif type(x) is list:
      #    out[name[:-1]] = tuple(x)
      #elif type(x) is list:
      #    i = 0
      #    for a in x:
      #        flatten(a, name + str(i) + '_')
      #        i += 1
      else:
          out[name[:-1]] = x

  flatten(nested_json)
  return out


flatten_json_2(data)

In [ ]:
 def flatten_json_3(nested_json):
  out = {}

  def flatten(x, name=''):
      if type(x) is dict:
          for a in x.keys():
              if a =='state':
                  out['game_id'] = x[a]['game_id']
              elif a== 'orders':
                  out['orders'] = x[a]
          #for a in x:
              #flatten(x[a], name + a + '_')
      #elif type(x) is list:
      #    out[name[:-1]] = tuple(x)
      #elif type(x) is list:
      #    i = 0
      #    for a in x:
      #        flatten(a, name + str(i) + '_')
      #        i += 1
      #else:
          #out[name[:-1]] = x

  flatten(nested_json)
  return out


flatten_json_3(data)